김서윤/2018311913/0.817907444668008

# 0. 데이터 로드

In [1]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 19.23 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [2]:
import pandas as pd

train_df = pd.read_csv('./nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('./nsmc/ratings_test.txt', sep='\t')
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [4]:
from nltk.tokenize import word_tokenize
train_text = []
test_text = []
for text in train_df['document'][:8000]:
  train_text.append(word_tokenize(text))
for text in test_df['document'][8000:9000]:
  test_text.append(word_tokenize(text))
train_label = train_df['label'][:8000]
test_label = test_df['label'][8000:9000]

# 1. 데이터 정제

먼저 단어 단위로 쪼개져있는 문장들을 하나의 문장(문자열)로 되돌린다.

In [5]:
for i in range(len(train_text)):
  train_text[i] = " ".join(train_text[i])

In [6]:
train_text[:5]

['아 더빙.. 진짜 짜증나네요 목소리',
 '흠 ... 포스터보고 초딩영화줄 ... .오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화 ! 스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']

특수문자나 영문자, 문장 부호 등을 모두 제거한다.

In [7]:
import re
def clean_text(data):
  clean_text = re.sub('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9]+\.[a-zA-Z0-9.]+)','',data)
  clean_text = re.sub('([a-zA-Z0-9_.+-]+\.[a-zA-Z0-9]+\.[a-zA-Z0-9.]+)','',clean_text)
  clean_text = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F{2}]))+','',clean_text)
  clean_text = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)','',clean_text)
  clean_text = re.sub('<[^>]*>...','',clean_text)
  clean_text = re.sub('[-=+_,#/\?..;:^$@*\"※~&%ㆍ!』\\‘’“”|\(\)\[\]\<\>`\'》❤♥♡☆★▲◇△▶◆■⊙▦ⓒ●◈·]',' ',clean_text)
  clean_text = clean_text.replace("\t"," ")
  clean_text = clean_text.replace("  "," ")
  clean_text = clean_text.replace("   "," ")
  clean_text = clean_text.replace("  "," ")

  return clean_text

In [8]:
for i in range(len(train_text)):
    train_text[i] = clean_text(train_text[i])

In [9]:
train_text[:20]

['아 더빙 진짜 짜증나네요 목소리',
 '흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 솔직히 재미는 없다 평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화 별반개도 아까움 ',
 '원작의 긴장감을 제대로 살려내지못했다 ',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지 정말 발로해도 그것보단 낫겟다 납치 감금만반복반복 이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데 꽤 볼만한데 헐리우드식 화려함에만 너무 길들여져 있나 ',
 '걍인피니트가짱이다 진짜짱이다 ',
 '볼때마다 눈물나서 죽겠다90년대의 향수자극 허진호는 감성절제멜로의 달인이다 ',
 '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해',
 '담백하고 깔끔해서 좋다 신문기사로만 보다 보면 자꾸 잊어버린다 그들도 사람이었다는 것을 ',
 '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지',
 '냥 매번 긴장되고 재밋음',
 '참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다 그냥 까고싶어서 안달난것처럼 보인다',
 '굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐',
 '이건 정말 깨알 캐스팅과 질퍽하지않은 산뜻한 내용구성이 잘 버무러진 깨알일드 ',
 '약탈자를 위한 변명 이라 저놈들은 착한놈들 절대 아닌걸요 ']

텍스트와 라벨을 합쳐 하나의 데이터프레임으로 만든다.


In [10]:
type(train_text)

list

In [11]:
train_label = list(train_label)

In [12]:
train_data = pd.DataFrame({"document":train_text, "label":train_label})

In [14]:
train_data.head()

,document,label
0,아 더빙 진짜 짜증나네요 목소리,0
1,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


전처리로 인해 제거되어 버린 리뷰들을 제거한다.

공백을 빈 값으로 설정한다.

In [15]:
train_data["document"] = train_data["document"].replace('^ +', "")

공백은 null 값으로 변경한다.

In [16]:
import numpy as np
train_data["document"].replace('', np.nan, inplace=True)

정제 후 null 값이 있는지 확인한다.

In [17]:
train_data.isnull().sum()

document    16
label        0
dtype: int64

 null 값을 가진 행을 제거한다.

In [18]:
train_data = train_data.dropna(how='any')

In [19]:
len(train_data)

7984

In [20]:
train_data.head()

,document,label
0,아 더빙 진짜 짜증나네요 목소리,0
1,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


test 데이터에도 train 데이터 전처리 과정을 똑같이 수행해준다.

In [21]:
for i in range(len(test_text)):
  test_text[i] = " ".join(test_text[i])

for i in range(len(test_text)):
    test_text[i] = clean_text(test_text[i])

test_label = list(test_label)
test_data = pd.DataFrame({"document":test_text, "label":test_label})
test_data["document"] = test_data["document"].replace('^ +', "")
import numpy as np
test_data["document"].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')
len(test_data)

994

In [22]:
test_data.head()

,document,label
0,팔황자 때문에 병남,1
1,옛날에 봤는데도 아직도 잊혀지지 않을만큼 강렬했던 한국 애니메이션이였다,1
2,꽝 10자이상 등록해 한다니 할수없이 쓴다 정말 꽝이다,0
3,수많은 영화 장면의 패러디만 있을 뿐 내용 면에선 너무 유치하다,0
4,1999년작 블레어윗치랑 같은 내용 오히려 긴장감은 더 떨어짐 볼만 했던 것은 붐마...,0


# 2. 토큰화

In [23]:
! pip install konlpy

     |████████████████████████████████| 19.4 MB 131 kB/s 
     |████████████████████████████████| 448 kB 41.9 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


한국어 불용어 사전을 참고하여 불용어를 정의해준다.
 https://www.ranks.nl/stopwords/korean 

In [24]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] 

In [25]:
s = """아
휴
아이구
아이쿠
아이고
어
나
우리
저희
따라
의해
을
를
에
의
가
으로
로
에게
뿐이다
의거하여
근거하여
입각하여
기준으로
예하면
예를 들면
예를 들자면
저
소인
소생
저희
지말고
하지마
하지마라
다른
물론
또한
그리고
비길수 없다
해서는 안된다
뿐만 아니라
만이 아니다
만은 아니다
막론하고
관계없이
그치지 않다
그러나
그런데
하지만
든간에
논하지 않다
따지지 않다
설사
비록
더라도
아니면
만 못하다
하는 편이 낫다
불문하고
향하여
향해서
향하다
쪽으로
틈타
이용하여
타다
오르다
제외하고
이 외에
이 밖에
하여야
비로소
한다면 몰라도
외에도
이곳
여기
부터
기점으로
따라서
할 생각이다
하려고하다
이리하여
그리하여
그렇게 함으로써
하지만
일때
할때
앞에서
중에서
보는데서
으로써
로써
까지
해야한다
일것이다
반드시
할줄알다
할수있다
할수있어
임에 틀림없다
한다면
등
등등
제
겨우
단지
다만
할뿐
딩동
댕그
대해서
대하여
대하면
훨씬
얼마나
얼마만큼
얼마큼
남짓
여
얼마간
약간
다소
좀
조금
다수
몇
얼마
지만
하물며
또한
그러나
그렇지만
하지만
이외에도
대해 말하자면
뿐이다
다음에
반대로
반대로 말하자면
이와 반대로
바꾸어서 말하면
바꾸어서 한다면
만약
그렇지않으면
까악
툭
딱
삐걱거리다
보드득
비걱거리다
꽈당
응당
해야한다
에 가서
각
각각
여러분
각종
각자
제각기
하도록하다
와
과
그러므로
그래서
고로
한 까닭에
하기 때문에
거니와
이지만
대하여
관하여
관한
과연
실로
아니나다를가
생각한대로
진짜로
한적이있다
하곤하였다
하
하하
허허
아하
거바
와
오
왜
어째서
무엇때문에
어찌
하겠는가
무슨
어디
어느곳
더군다나
하물며
더욱이는
어느때
언제
야
이봐
어이
여보시오
흐흐
흥
휴
헉헉
헐떡헐떡
영차
여차
어기여차
끙끙
아야
앗
아야
콸콸
졸졸
좍좍
뚝뚝
주룩주룩
솨
우르르
그래도
또
그리고
바꾸어말하면
바꾸어말하자면
혹은
혹시
답다
및
그에 따르는
때가 되어
즉
지든지
설령
가령
하더라도
할지라도
일지라도
지든지
몇
거의
하마터면
인젠
이젠
된바에야
된이상
만큼	어찌됏든
그위에
게다가
점에서 보아
비추어 보아
고려하면
하게될것이다
일것이다
비교적
좀
보다더
비하면
시키다
하게하다
할만하다
의해서
연이서
이어서
잇따라
뒤따라
뒤이어
결국
의지하여
기대여
통하여
자마자
더욱더
불구하고
얼마든지
마음대로
주저하지 않고
곧
즉시
바로
당장
하자마자
밖에 안된다
하면된다
그래
그렇지
요컨대
다시 말하자면
바꿔 말하면
즉
구체적으로
말하자면
시작하여
시초에
이상
허
헉
허걱
바와같이
해도좋다
해도된다
게다가
더구나
하물며
와르르
팍
퍽
펄렁
동안
이래
하고있었다
이었다
에서
로부터
까지
예하면
했어요
해요
함께
같이
더불어
마저
마저도
양자
모두
습니다
가까스로
하려고하다
즈음하여
다른
다른 방면으로
해봐요
습니까
했어요
말할것도 없고
무릎쓰고
개의치않고
하는것만 못하다
하는것이 낫다
매
매번
들
모
어느것
어느
로써
갖고말하자면
어디
어느쪽
어느것
어느해
어느 년도
라 해도
언젠가
어떤것
어느것
저기
저쪽
저것
그때
그럼
그러면
요만한걸
그래
그때
저것만큼
그저
이르기까지
할 줄 안다
할 힘이 있다
너
너희
당신
어찌
설마
차라리
할지언정
할지라도
할망정
할지언정
구토하다
게우다
토하다
메쓰겁다
옆사람
퉤
쳇
의거하여
근거하여
의해
따라
힘입어
그
다음
버금
두번째로
기타
첫번째로
나머지는
그중에서
견지에서
형식으로 쓰여
입장에서
위해서
단지
의해되다
하도록시키다
뿐만아니라
반대로
전후
전자
앞의것
잠시
잠깐
하면서
그렇지만
다음에
그러한즉
그런즉
남들
아무거나
어찌하든지
같다
비슷하다
예컨대
이럴정도로
어떻게
만약
만일
위에서 서술한바와같이
인 듯하다
하지 않는다면
만약에
무엇
무슨
어느
어떤
아래윗
조차
한데
그럼에도 불구하고
여전히
심지어
까지도
조차도
하지 않도록
않기 위하여
때
시각
무렵
시간
동안
어때
어떠한
하여금
네
예
우선
누구
누가 알겠는가
아무도
줄은모른다
줄은 몰랏다
하는 김에
겸사겸사
하는바
그런 까닭에
한 이유는
그러니
그러니까
때문에
그
너희
그들
너희들
타인
것
것들
너
위하여
공동으로
동시에
하기 위하여
어찌하여
무엇때문에
붕붕
윙윙
나
우리
엉엉
휘익
윙윙
오호
아하
어쨋든
만 못하다	하기보다는
차라리
하는 편이 낫다
흐흐
놀라다
상대적으로 말하자면
마치
아니라면
쉿
그렇지 않으면
그렇지 않다면
안 그러면
아니었다면
하든지
아니면
이라면
좋아
알았어
하는것도
그만이다
어쩔수 없다
하나
일
일반적으로
일단
한켠으로는
오자마자
이렇게되면
이와같다면
전부
한마디
한항목
근거로
하기에
아울러
하지 않도록
않기 위해서
이르기까지
이 되다
로 인하여
까닭으로
이유만으로
이로 인하여
그래서
이 때문에
그러므로
그런 까닭에
알 수 있다
결론을 낼 수 있다
으로 인하여
있다
어떤것
관계가 있다
관련이 있다
연관되다
어떤것들
에 대해
이리하여
그리하여
여부
하기보다는
하느니
하면 할수록
운운
이러이러하다
하구나
하도다
다시말하면
다음으로
에 있다
에 달려 있다
우리
우리들
오히려
하기는한데
어떻게
어떻해
어찌됏어
어때
어째서
본대로
자
이
이쪽
여기
이것
이번
이렇게말하자면
이런
이러한
이와 같은
요만큼
요만한 것
얼마 안 되는 것
이만큼
이 정도의
이렇게 많은 것
이와 같다
이때
이렇구나
것과 같이
끼익
삐걱
따위
와 같은 사람들
부류의 사람들
왜냐하면
중의하나
오직
오로지
에 한하다
하기만 하면
도착하다
까지 미치다
도달하다
정도에 이르다
할 지경이다
결과에 이르다
관해서는
여러분
하고 있다
한 후
혼자
자기
자기집
자신
우에 종합한것과같이
총적으로 보면
총적으로 말하면
총적으로
대로 하다
으로서
참
그만이다
할 따름이다
쿵
탕탕
쾅쾅
둥둥
봐
봐라
아이야
아니
와아
응
아이
참나
년
월
일
령
영
일
이
삼
사
오
육
륙
칠
팔
구
이천육
이천칠
이천팔
이천구
하나
둘
셋
넷
다섯
여섯
일곱
여덟
아홉
령
영"""

In [26]:
s = s.split("\n")

In [27]:
stopwords = stopwords+s

In [28]:
stopwords = set(stopwords) # 중복값 제거
stopwords = list(stopwords)

In [29]:
stopwords[:5]

['제외하고', '다시 말하자면', '그들', '그치지 않다', '의거하여']

토큰화 함수 정의

In [30]:
# twitter 토크나이저
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(sentence):
  tokens_tw = twitter.morphs(sentence, stem=True)
  return tokens_tw

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [31]:
# Okt 토크나이저
from konlpy.tag import Okt
okt = Okt()
def ok_tokenizer(sentence):
  tokens_ok = okt.morphs(sentence, stem = True)
  return tokens_ok

# 3. 벡터화

In [32]:
X_train = train_data["document"]
y_train = train_data["label"]
X_test = test_data["document"]
y_test = test_data["label"]

In [37]:
# 1-1 Count Vextorizer + tw_tokenizser
# F1 = 0.817306686623605
# accuracy = 0.817907444668008
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer(tokenizer = tw_tokenizer, stop_words=stopwords, ngram_range=(1,2))
X_train_vec_tw = cnt_vect.fit_transform(X_train)
X_test_vec_tw = cnt_vect.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\t', '가다', '각하', '간', '갖다', '개', '거나', '거리', '걸', '겁', '것과', '견지', '결과', '결론', '고려', '곤', '곳', '공동', '관', '관계', '관련', '구체', '구토', '그러하다', '그런', '그렇게', '그렇다', '그치다', '근거', '기대', '기점', '기준', '김', '까닭', '나머지', '남', '낫다', '낼', '년도', '논', '누가', '늘다', '다', '다르다', '다시', '달다', '답', '대로', '대하', '대해', '더', '더불다', '더욱이', '도달', '도착', '동시', '됏', '되다', '되어다', '두번째', '뒤', '뒤따르다', '든', '든지', '들다', '듯', '따다', '따르다', '따름', '때문', '라', '라도', '라면', '리하', '마자', '막론', '만', '만으로', '만은', '만이', '만큼', '만하', '많다', '말', '말다', '망정', '메', '면', '모르다', '몰랏다', '못', '무릎', '미치다', '바', '바꾸다', '밖에', '반대', '방면', '번째', '보다', '보아', '본대', '부류', '불구', '불문', '비', '비걱거리', '비교', '비기다', '비추다', '뿐', '사람', '상대', '생각', '서', '서술', '수', '시오', '시작', '시초', '쓰다', '쓰이다', '아니다', '아래', '아무', '아우르다', '안', '안다', '안되다', '않다', '알', '알다', '앞', '어기', '어떠

In [38]:
# 1-2 Count Vextorizer + ok_tokenizer
# F1 = 0.817306686623605
# accuracy = 0.817907444668008
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer(tokenizer = ok_tokenizer, stop_words=stopwords, ngram_range=(1,2))
X_train_vec_ok = cnt_vect.fit_transform(X_train)
X_test_vec_ok = cnt_vect.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\t', '가다', '각하', '간', '갖다', '개', '거나', '거리', '걸', '겁', '것과', '견지', '결과', '결론', '고려', '곤', '곳', '공동', '관', '관계', '관련', '구체', '구토', '그러하다', '그런', '그렇게', '그렇다', '그치다', '근거', '기대', '기점', '기준', '김', '까닭', '나머지', '남', '낫다', '낼', '년도', '논', '누가', '늘다', '다', '다르다', '다시', '달다', '답', '대로', '대하', '대해', '더', '더불다', '더욱이', '도달', '도착', '동시', '됏', '되다', '되어다', '두번째', '뒤', '뒤따르다', '든', '든지', '들다', '듯', '따다', '따르다', '따름', '때문', '라', '라도', '라면', '리하', '마자', '막론', '만', '만으로', '만은', '만이', '만큼', '만하', '많다', '말', '말다', '망정', '메', '면', '모르다', '몰랏다', '못', '무릎', '미치다', '바', '바꾸다', '밖에', '반대', '방면', '번째', '보다', '보아', '본대', '부류', '불구', '불문', '비', '비걱거리', '비교', '비기다', '비추다', '뿐', '사람', '상대', '생각', '서', '서술', '수', '시오', '시작', '시초', '쓰다', '쓰이다', '아니다', '아래', '아무', '아우르다', '안', '안다', '안되다', '않다', '알', '알다', '앞', '어기', '어떠

In [39]:
# 2-1 Tf-idf vectorizer + tw_tokenizer
# F1 = 0.8073399057269125
# accuracy = 0.8078470824949698
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(tokenizer = tw_tokenizer, stop_words=stopwords, ngram_range=(1,2))
X_train_tf_tw = tf_idf.fit_transform(X_train)
X_test_tf_tw = tf_idf.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\t', '가다', '각하', '간', '갖다', '개', '거나', '거리', '걸', '겁', '것과', '견지', '결과', '결론', '고려', '곤', '곳', '공동', '관', '관계', '관련', '구체', '구토', '그러하다', '그런', '그렇게', '그렇다', '그치다', '근거', '기대', '기점', '기준', '김', '까닭', '나머지', '남', '낫다', '낼', '년도', '논', '누가', '늘다', '다', '다르다', '다시', '달다', '답', '대로', '대하', '대해', '더', '더불다', '더욱이', '도달', '도착', '동시', '됏', '되다', '되어다', '두번째', '뒤', '뒤따르다', '든', '든지', '들다', '듯', '따다', '따르다', '따름', '때문', '라', '라도', '라면', '리하', '마자', '막론', '만', '만으로', '만은', '만이', '만큼', '만하', '많다', '말', '말다', '망정', '메', '면', '모르다', '몰랏다', '못', '무릎', '미치다', '바', '바꾸다', '밖에', '반대', '방면', '번째', '보다', '보아', '본대', '부류', '불구', '불문', '비', '비걱거리', '비교', '비기다', '비추다', '뿐', '사람', '상대', '생각', '서', '서술', '수', '시오', '시작', '시초', '쓰다', '쓰이다', '아니다', '아래', '아무', '아우르다', '안', '안다', '안되다', '않다', '알', '알다', '앞', '어기', '어떠

In [40]:
# 2-2 Tf-idf vectorizer + ok_tokenizer
# F1 = 0.8073399057269125
# accuracy = 0.8078470824949698
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(tokenizer = ok_tokenizer, stop_words=stopwords, ngram_range=(1,2))
X_train_tf_ok = tf_idf.fit_transform(X_train)
X_test_tf_ok = tf_idf.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\t', '가다', '각하', '간', '갖다', '개', '거나', '거리', '걸', '겁', '것과', '견지', '결과', '결론', '고려', '곤', '곳', '공동', '관', '관계', '관련', '구체', '구토', '그러하다', '그런', '그렇게', '그렇다', '그치다', '근거', '기대', '기점', '기준', '김', '까닭', '나머지', '남', '낫다', '낼', '년도', '논', '누가', '늘다', '다', '다르다', '다시', '달다', '답', '대로', '대하', '대해', '더', '더불다', '더욱이', '도달', '도착', '동시', '됏', '되다', '되어다', '두번째', '뒤', '뒤따르다', '든', '든지', '들다', '듯', '따다', '따르다', '따름', '때문', '라', '라도', '라면', '리하', '마자', '막론', '만', '만으로', '만은', '만이', '만큼', '만하', '많다', '말', '말다', '망정', '메', '면', '모르다', '몰랏다', '못', '무릎', '미치다', '바', '바꾸다', '밖에', '반대', '방면', '번째', '보다', '보아', '본대', '부류', '불구', '불문', '비', '비걱거리', '비교', '비기다', '비추다', '뿐', '사람', '상대', '생각', '서', '서술', '수', '시오', '시작', '시초', '쓰다', '쓰이다', '아니다', '아래', '아무', '아우르다', '안', '안다', '안되다', '않다', '알', '알다', '앞', '어기', '어떠

두 토크나이저의 성능 차이는 없다. 다만 tf-idf 벡터라이저를 사용했을 때 점수가 조금 더 높다.

# 4. 모델링

로지스틱 리그레션

In [41]:
# 1 트위터 + 카운트벡터라이저 + 로지스틱리그레션
from sklearn.linear_model import LogisticRegression #변경 가능
from sklearn import metrics
clf = LogisticRegression(random_state=500)
clf.fit(X_train_vec_tw, y_train) #변경 가능
pred = clf.predict(X_test_vec_tw) #변경 가능
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))

F1 = 0.817306686623605
accuracy = 0.817907444668008


In [42]:
# 1 Okt + 카운트벡터라이저 + 로지스틱리그레션
from sklearn.linear_model import LogisticRegression #변경 가능
from sklearn import metrics
clf = LogisticRegression(random_state=500)
clf.fit(X_train_vec_ok, y_train) #변경 가능
pred = clf.predict(X_test_vec_ok) #변경 가능
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))

F1 = 0.817306686623605
accuracy = 0.817907444668008


In [43]:
# 2 트위터 + TfIdf 벡터라이저 + 로지스틱리그레션
from sklearn.linear_model import LogisticRegression #변경 가능
from sklearn import metrics
clf = LogisticRegression(random_state=500)
clf.fit(X_train_tf_tw, y_train) #변경 가능
pred = clf.predict(X_test_tf_tw) #변경 가능
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))

F1 = 0.8073399057269125
accuracy = 0.8078470824949698


In [44]:
# 2 Okt + TfIdf 벡터라이저 + 로지스틱리그레션
from sklearn.linear_model import LogisticRegression #변경 가능
from sklearn import metrics
clf = LogisticRegression(random_state=500)
clf.fit(X_train_tf_ok, y_train) #변경 가능
pred = clf.predict(X_test_tf_ok) #변경 가능
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))

F1 = 0.8073399057269125
accuracy = 0.8078470824949698


XGBoost

In [103]:
import xgboost as xgb 
from sklearn.model_selection import KFold, GridSearchCV
from sklearn import metrics

# K-fold
cv = KFold(n_splits=5, random_state=None)
# 모델 설정
clf = xgb.XGBClassifier()

# 초매개변수 튜닝(CV, Grid Search)
param_grid={
    'objective':['binary:logistic'],
    'max_depth':[5],
    'learning_rate':[0.1],
    'gamma':[0.25],
    'reg_lambda':[1.0],
    'min_child_weight':[3],
    'n_estimators':[500]    
    }

grid = GridSearchCV(estimator = clf, param_grid = param_grid, cv = cv)
grid_fit = grid.fit(X_train_tf, y_train)

# 최적 파라미터 구함
print("final params", grid.best_params_)

final params {'gamma': 0.25, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 500, 'objective': 'binary:logistic', 'reg_lambda': 1.0}


In [104]:
#### 최적 파라미터 적용한 모델 #### : best_model
best_model = grid_fit.best_estimator_
# Y값 예측
pred = best_model.predict(X_test_vec)

In [105]:
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))

F1 = 0.7532151546750087
accuracy = 0.7535211267605634


원래 코드: 나이브베이즈

In [50]:
from sklearn.naive_bayes import MultinomialNB #변경 가능
from sklearn import metrics
clf = MultinomialNB(alpha=.01)
clf.fit(X_train_vec_tw, y_train) #변경 가능
pred = clf.predict(X_test_vec_tw) #변경 가능
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))

F1 = 0.7917820811623206
accuracy = 0.7927565392354124


# 5. 최종 모델

### F1 = 0.817306686623605
### accuracy = 0.817907444668008

In [52]:
# twitter 토크나이저
from konlpy.tag import Twitter

twitter = Twitter()
def tw_tokenizer(sentence):
  tokens_tw = twitter.morphs(sentence, stem=True)
  return tokens_tw

# Count Vextorizer + tw_tokenizser
# F1 = 0.817306686623605
# accuracy = 0.817907444668008
from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer(tokenizer = tw_tokenizer, stop_words=stopwords, ngram_range=(1,2))
X_train_vec_tw = cnt_vect.fit_transform(X_train)
X_test_vec_tw = cnt_vect.transform(X_test)

# 1 트위터 + 카운트벡터라이저 + 로지스틱리그레션
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression(random_state=500)
clf.fit(X_train_vec_tw, y_train)
pred = clf.predict(X_test_vec_tw)
print('F1 =', metrics.f1_score(y_test, pred, average='macro'))
print('accuracy =', metrics.accuracy_score(y_test, pred))

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['\t', '가다', '각하', '간', '갖다', '개', '거나', '거리', '걸', '겁', '것과', '견지', '결과', '결론', '고려', '곤', '곳', '공동', '관', '관계', '관련', '구체', '구토', '그러하다', '그런', '그렇게', '그렇다', '그치다', '근거', '기대', '기점', '기준', '김', '까닭', '나머지', '남', '낫다', '낼', '년도', '논', '누가', '늘다', '다', '다르다', '다시', '달다', '답', '대로', '대하', '대해', '더', '더불다', '더욱이', '도달', '도착', '동시', '됏', '되다', '되어다', '두번째', '뒤', '뒤따르다', '든', '든지', '들다', '듯', '따다', '따르다', '따름', '때문', '라', '라도', '라면', '리하', '마자', '막론', '만', '만으로', '만은', '만이', '만큼', '만하', '많다', '말', '말다', '망정', '메', '면', '모르다', '몰랏다', '못', '무릎', '미치다', '바', '바꾸다', '밖에', '반대', '방면', '번째', '보다', '보아', '본대', '부류', '불

F1 = 0.817306686623605
accuracy = 0.817907444668008
